In [ ]:
import retro


In [ ]:


def play_game():
    global env
    obs = env.reset()
    totalFrames = 0
    totalReward = 0
    isPlaying = True
    while isPlaying:
        env.render()

        action, _state = model.predict(observation=obs)
        obs, reward, done, info = env.step(action)

        totalReward += reward
        totalFrames += 1

        if totalFrames >= 5_000 and totalReward < 7:
            isPlaying = False
        elif done:
            isPlaying = False

        print(totalFrames, totalReward, reward)
    env.close()
    return totalReward

## preprocess env

In [ ]:

from gym.wrappers import GrayScaleObservation

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

from matplotlib import pyplot as plt

In [ ]:
# 1. create base env
env = retro.make(game='HotWheelsStuntTrackChallenge-GameBoy')
state = env.reset()
print(state.shape, env.action_space)

In [ ]:
# 2. grayscale env (keep original dimentions)
env = GrayScaleObservation(env, keep_dim=True)
state = env.reset()
state.shape

In [ ]:
# 3. Wrap in dummy env
env = DummyVecEnv([lambda: env])
state = env.reset()
state.shape

In [ ]:
# 4. stack frames
env = VecFrameStack(env, n_stack=4, channels_order='last')
state = env.reset()
state.shape

## Train model

#### PPO CNN Policy

In [ ]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# TODO: Modyify to work in colab

# class to save model every 10k steps
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [ ]:
CHECKPOINT_DIR = './checkpoint/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=50_000, save_path=CHECKPOINT_DIR)

In [ ]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
model.learn(total_timesteps=100_000, callback=callback, progress_bar=True)

In [ ]:
model = PPO.load('./checkpoint/__XYZ')